#### **Analyzing CIA FactBook Data Using SQL**
![CIA World Factbook](./cia.png)


- [1 Introduction](#1)
- [2 Analysis on human population and its spread](#2)
- [3 Analysis of Area v/s Population](#3)
- [4 Exploring Birth and Death rates](#4)
- [5 Analysis of Land v/s Water](#5)
- [6 Conclusion](#6)

##### **Connecting to SQLite DataBase**

In [1]:
%load_ext sql
%sql sqlite:///factbook.db

# 1

#### **Introduction to the Data**

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The dataset used for this analysis can be found [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db). The Factbook contains demographic information like the following:

* **population** — the global population.
* **population_growth** — the annual population growth rate, as a percentage.
* **area** — the total land and water area.


##### **Overview of the Data**

Query the database to get information of the tables.

In [2]:
%%sql
SELECT * 
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* **name** — the name of the country.
* **area**— the country's total area (both land and water).
* **area_land** — the country's land area in square kilometers.
* **area_water** — the country's waterarea in square kilometers.
* **population** — the country's population.
* **population_growth** — the country's population growth as a percentage.
* **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.
* **death_rate** — the country's death rate, or the number of death per year per 1,000 people.


# 2

#### **Analysis on human population and its spread**

##### **Summary Statistics**

In [4]:
%%sql
SELECT  MIN(population), 
        MAX(population),
	    MIN(population_growth), 
        MAX(population_growth)
FROM facts;	   

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

* There's a country with a population of **0**
* There's a country with a population of **7256490011 (or more than 7.2 billion people)**


#### **Exploring Outliers**

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [Page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html)

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We also see that the table contains a row for the **whole world**, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

#### **Summary Statistics Revisited**

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

In [8]:
%%sql
SELECT name AS 'Most Populous Country', 
       population AS 'Population', 
       population_growth AS 'Population Growth Rate'
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts
                    WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


Most Populous Country,Population,Population Growth Rate
China,1367485388,0.45


While China is the most populous country in the world. Its growth rate seems quite low. Let's find out the population of the country with the highest growth rate.

In [9]:
%%sql
SELECT name AS 'Country with highest population growth rate', 
       population AS 'Population', 
       population_growth AS 'Population Growth Rate'
FROM facts
WHERE population_growth = (SELECT MAX(population_growth)
                    FROM facts
                    WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


Country with highest population growth rate,Population,Population Growth Rate
South Sudan,12042910,4.02


**Which country has the most people? Which country has the highest growth rate?**

While China has a population that is 100 times more than South Sudan, its population growth rate is only a tenth of that of South Sudan. This raises the question of whether South Sudan might be an outlier with regards to population growth rate. An average of the population growth rate of the world should reveal a better understanding.

In [10]:
%%sql 
SELECT ROUND(AVG(population_growth),3) AS 'Average Population Growth'
FROM facts
WHERE name<>'World'

 * sqlite:///factbook.db
Done.


Average Population Growth
1.201


South Sudan's growth rate seems to be higher than average. It would be interesting to find out how many countries have above average growth rates.

In [11]:
%%sql
SELECT COUNT(*) AS 'Number of countries with more than average population growth'
FROM facts
WHERE population_growth > (SELECT AVG(population_growth)
                           FROM facts
                           WHERE name<>'World')

 * sqlite:///factbook.db
Done.


Number of countries with more than average population growth
101


In [12]:
%%sql
SELECT name AS 'Country Name', population_growth AS 'Population Growth Rate'
FROM facts
WHERE population_growth>(SELECT AVG(population_growth)
                         FROM facts)
ORDER BY population_growth DESC
LIMIT 30

 * sqlite:///factbook.db
Done.


Country Name,Population Growth Rate
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


More than a fifth of the countries in the above list are in Africa. 

# 3

#### **Exploring Average Population And Area**
Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [13]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

#### **Finding Densely Populated Countries**

To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

* Above average values for population.
* Below average values for area.

In [14]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are generally known to be densely populated, so we have confidence in our results!

In [15]:
%%sql
--#Densely Populated countries in descending order of area to population ratio
SELECT name AS Country, ROUND((CAST(area AS FLOAT)/population),5) AS 'Area/Population_Ratio', population AS Population, area AS Area
FROM facts
WHERE (population > (SELECT AVG(population)
                    FROM facts
                    WHERE name!= 'World')
       AND
       area < (SELECT AVG(area)
               FROM facts
               WHERE name!='World')
      )
ORDER BY population/area DESC

 * sqlite:///factbook.db
Done.


Country,Area/Population_Ratio,Population,Area
Bangladesh,0.00088,168957745,148460
"Korea, South",0.00203,49115196,99720
Philippines,0.00297,100998376,300000
Japan,0.00298,126919659,377915
Vietnam,0.00351,94348835,331210
United Kingdom,0.0038,64088222,243610
Germany,0.00442,80854408,357022
Italy,0.00487,61855120,301340
Uganda,0.0065,37101745,241038
Thailand,0.00755,67976405,513120


Bangladesh has been identified as the most densely populated country in the world.

# 4

#### **Exploring birth and death rates**

Birth rates are good indicators for population growth rates.

In [16]:
%%sql
SELECT ROUND(AVG(birth_rate), 3) AS 'Average Birth Rate'
FROM facts

 * sqlite:///factbook.db
Done.


Average Birth Rate
19.329


In [17]:
%%sql
SELECT name AS Country, 
       birth_rate AS 'Birth Rate', 
       death_rate AS 'Death Rate'      
FROM facts
WHERE birth_rate > death_rate
ORDER BY birth_rate - death_rate DESC
LIMIT 30

 * sqlite:///factbook.db
Done.


Country,Birth Rate,Death Rate
Malawi,41.56,8.41
Uganda,43.79,10.69
Niger,45.45,12.42
Burundi,42.01,9.27
Mali,44.99,12.89
Burkina Faso,42.03,11.72
Zambia,42.13,12.67
Ethiopia,37.27,8.19
South Sudan,36.91,8.18
Tanzania,36.39,8.0


The above confirms our understanding on growth rate. Many of the countries displayed are from **Africa** and have a birth rate that is higher than the global average.

This leads to a need to find countries that are at the other end of the spectrum. Countries that have a death rate that is greater than their birth rate.

In [18]:
%%sql
SELECT name AS Country, 
       birth_rate AS 'Birth Rate', 
       death_rate AS 'Death Rate'       
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate-birth_rate DESC

 * sqlite:///factbook.db
Done.


Country,Birth Rate,Death Rate
Bulgaria,8.92,14.44
Serbia,9.08,13.66
Latvia,10.0,14.31
Lithuania,10.1,14.27
Ukraine,10.72,14.46
Hungary,9.16,12.73
Germany,8.47,11.42
Slovenia,8.42,11.37
Romania,9.14,11.9
Croatia,9.45,12.18


# 5

#### **Analysis of Land v/s Water**

**Countries with more water than land**

There are countries around the world where water covers more area than the land.

In [19]:
%%sql
SELECT name, area_water, area_land 
FROM facts
WHERE area_water > area_land
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


Only two countries have a greater water to land ratio

##### **Which countries have the highest ratios of water to land?**

In [20]:
%%sql

SELECT name AS 'Top 10', ROUND(CAST(area_water AS FLOAT)/area_land,3) AS 'Water Area/Land Area'
FROM facts
ORDER BY CAST(area_water AS FLOAT)/area_land DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


Top 10,Water Area/Land Area
British Indian Ocean Territory,905.667
Virgin Islands,4.52
Puerto Rico,0.555
"Bahamas, The",0.387
Guinea-Bissau,0.285
Malawi,0.259
Netherlands,0.226
Uganda,0.223
Eritrea,0.164
Liberia,0.156


# 6

#### **Conclusion**

The **CIA World Factbook** is a collection of demographic, geographical and other related facts of countries around the world. Based on the provided information we have undertaken analysis on a small set of data. The data was read and we've come up with many insights. Population growth is distributed variedly around the world. Population, however is not a good indicator of population growth. Birth rates, on the other hand are a good indicator of population growth.

Many African countries are projected to have a larger population over the coming years.